In [6]:
import numpy as np
import pandas as pd

import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.nn import functional as F

from torch.utils.data import TensorDataset, DataLoader

sns.set(style="darkgrid", font_scale=1.4)

import os
from torchvision.datasets import MNIST
from torchvision import transforms as tfs
from torchsummary import summary

device = "cuda" if torch.cuda.is_available() else "cpu"

data_tfs = tfs.Compose([
    tfs.ToTensor(),
    tfs.Normalize((0.5), (0.5))
])


# install for train and test
root = './'
train_dataset = MNIST(root, train=True,  transform=data_tfs, download=True)
val_dataset  = MNIST(root, train=False, transform=data_tfs, download=True)

batch_size = 128

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, drop_last=True)
valid_dataloader = DataLoader(val_dataset, batch_size=batch_size, drop_last=True)

activation = nn.ELU
features = 784  # number of flattened array
classes = 10    # number of classes

model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(features, 128),
    activation(),
    nn.Linear(128, 128),
    activation(),
    nn.Linear(128, classes),
    # don't need softmax here!
)
model.to(device)
print(device)
summary(model, (features,), batch_size=batch_size)

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                 [128, 784]               0
            Linear-2                 [128, 128]         100,480
               ELU-3                 [128, 128]               0
            Linear-4                 [128, 128]          16,512
               ELU-5                 [128, 128]               0
            Linear-6                  [128, 10]           1,290
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.38
Forward/backward pass size (MB): 1.28
Params size (MB): 0.45
Estimated Total Size (MB): 2.11
----------------------------------------------------------------


In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

loaders = {"train": train_dataloader, "valid": valid_dataloader}
max_epochs = 10
accuracy = {"train": [], "valid": []}
for epoch in range(max_epochs):
    for k, dataloader in loaders.items():
        epoch_correct = 0
        epoch_all = 0
        for x_batch, y_batch in dataloader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            if k == "train":
                model.train()
                optimizer.zero_grad()
                outp = model(x_batch)
            else:
                model.eval()
                with torch.no_grad():
                  outp = model(x_batch)
            preds = outp.argmax(-1)
            correct = (preds == y_batch).sum()
            all =  len(y_batch)
            epoch_correct += correct
            epoch_all += all
            if k == "train":
                loss = criterion(outp, y_batch)
                loss.backward()
                optimizer.step()
        if k == "train":
            print(f"Epoch: {epoch+1}")
        print(f"Loader: {k}. Accuracy: {epoch_correct/epoch_all}")
        accuracy[k].append(epoch_correct/epoch_all)


Epoch: 1
Loader: train. Accuracy: 0.8780048489570618
Loader: valid. Accuracy: 0.9273838400840759
Epoch: 2
Loader: train. Accuracy: 0.9411725997924805
Loader: valid. Accuracy: 0.9491186141967773
Epoch: 3
Loader: train. Accuracy: 0.9594685435295105
Loader: valid. Accuracy: 0.9564303159713745
Epoch: 4
Loader: train. Accuracy: 0.9686498641967773
Loader: valid. Accuracy: 0.9640424847602844
Epoch: 5
Loader: train. Accuracy: 0.9743590354919434
Loader: valid. Accuracy: 0.9672476053237915
Epoch: 6
Loader: train. Accuracy: 0.9792001247406006
Loader: valid. Accuracy: 0.9639423489570618
Epoch: 7
Loader: train. Accuracy: 0.9825554490089417
Loader: valid. Accuracy: 0.9639423489570618
Epoch: 8
Loader: train. Accuracy: 0.984408438205719
Loader: valid. Accuracy: 0.9640424847602844
Epoch: 9
Loader: train. Accuracy: 0.9850093722343445
Loader: valid. Accuracy: 0.9621394276618958
Epoch: 10
Loader: train. Accuracy: 0.9859775900840759
Loader: valid. Accuracy: 0.9704527258872986
